In [12]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
import pandas as pd
import yaml
import geopandas as gp
import time
import pandas as pd
from geopy.distance import great_circle
import folium
import h5py
from scipy.signal import butter, lfilter

from scipy.spatial import ConvexHull
from scipy.interpolate import griddata
from shapely.geometry import Point, Polygon
import csv

In [13]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import maplib
from toollib_standard import mathlib
from toollib_standard import filelib
from toollib_standard import stacklib
from toollib_standard import plotlib

In [14]:
flag_project = 1

if flag_project == 0:
    file_project = 'a-project.yml'
elif flag_project == 1:
    file_project = 'a-project_repar.yml'
elif flag_project == 2:
    file_project = 'a-project_voro.yml'
    
with open(file_project, 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
name_project = proj['name']
#name_project = 'project_repartition_v4.0/output_repar_v9.5_01--10-16Hz/'

In [15]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  ./
dir_project_workspace:  ./
dir_project:  ./project_repartition_v4.0/output_repar_v9.5_02--10-16Hz/


In [16]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [37]:
dir_CC = dir_CC_workspace + info_basic['rdir_CC']
dir_ds = dir_project + info_basic['rdir_ds']
dir_disp = dir_project + info_basic['rdir_disp_model']
dir_partition = dir_project + info_basic['rdir_partition']
dir_disp_pick = dir_project + info_basic['rdir_disp_pick']
dir_inv = dir_project + info_basic['rdir_inv_BFGS']
dir_surf = dir_project + info_basic['rdir_image'] + 'surf/'
if not os.path.exists(dir_surf):
    os.makedirs(dir_surf)
dir_input = dir_surf + 'input/'
if not os.path.exists(dir_input):
    os.makedirs(dir_input)

In [18]:
key_subworks = info_basic['key_subworks']

In [19]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
stalist_all = stainfo['Station'].tolist() 
lat_stations_all =  stainfo['latitude'].tolist() 
lon_stations_all =  stainfo['longitude'].tolist() 
elevations_all =  stainfo['elevation'].tolist()

In [20]:
stations_partition = {}
lat_stations_partition = {}
lon_stations_partition = {}
lat_centroid_partition = []
lon_centroid_partition = []
for key in info_basic['key_subworks']:
    filepath = dir_partition + str(key) + '.txt'
    stations_this, lat_stations_this, lon_stations_this = np.loadtxt(filepath, dtype='str' , unpack=True)
    stations_partition[key] = stations_this
    lat_stations_partition[key] = lat_stations_this.astype(float)
    lon_stations_partition[key] = lon_stations_this.astype(float)
    lat_centroid_partition.append(np.mean(lat_stations_this.astype(float)))
    lon_centroid_partition.append(np.mean(lon_stations_this.astype(float)))

In [24]:
inter_file = dir_inv + 'vs_inter.npz'
inter = np.load(inter_file, allow_pickle=True)
vs_fund = inter['vs_fund']
vs = inter['vs']
x_inter_in = inter['x']
y_inter_in = inter['y']
lon_inter_in = inter['lon']
lat_inter_in = inter['lat']
z = inter['depth']
dz = inter['dz']
N = inter['N']

### Generate csv for stalist

In [22]:
headers = ['stations','longitude','latitude','elevation']
filename = dir_input + 'stations.csv'
with open(filename,'w',newline='') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    for i in range(len(stalist_all)):
        f_csv.writerow([stalist_all[i],lon_stations_all[i],lat_stations_all[i],elevations_all[i]])

In [23]:
headers = ['partition','longitude','latitude']
filename = dir_input + 'partition.csv'
with open(filename,'w',newline='') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    for i in range(len(lon_centroid_partition)):
        f_csv.writerow([key_subworks[i],lon_centroid_partition[i],lat_centroid_partition[i]])

In [45]:
for i in range(N):
    depth = i*dz
    headers = ['x','y','longitude','latitude','depth','vs','vs_fund']
    filename = dir_input + 'vs_'+str(depth)+'.csv'
    with open(filename,'w',newline='') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
        for j in range(len(lon_inter_in)):
            f_csv.writerow([x_inter_in[j],y_inter_in[j],lon_inter_in[j],lat_inter_in[j],depth,vs[i,j],vs_fund[i,j]])

### Phase

In [38]:
f_list = [4,8,15]
phase = {}
for f in f_list:
    phase[f] = []
for key in info_basic['key_subworks']:
    disp = np.loadtxt(dir_disp + 'disp_model_'+key+'.txt')
    disp = disp[disp[:,2] == 0]
    for f in f_list:
        phase[f].append(disp[disp[:,0]==f][0][1])

In [43]:
headers = ['longitude','latitude','4Hz','8Hz','15Hz']
filename = dir_input + 'phase.csv'
with open(filename,'w',newline='') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    for i in range(len(lon_centroid_partition)):
        f_csv.writerow([lon_centroid_partition[i],lat_centroid_partition[i],phase[4][i],phase[8][i],phase[15][i]])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

### Faults

In [53]:
# fault surface trace
faults = np.load(dir_project +'clark_faults_trans.npy', allow_pickle='TRUE').item()
fault_infer = np.load(dir_project +'clark_fault_inferred_trans.npy', allow_pickle='TRUE').item()

In [61]:
for i in range(len(faults)):
    headers = ['longitude','latitude']
    filename = dir_input + 'fault_'+str(i)+'.csv'
    with open(filename,'w',newline='') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
        for k in range(len(faults['clark'+str(i+1)]['x'])):
            f_csv.writerow([faults['clark'+str(i+1)]['x'][k],faults['clark'+str(i+1)]['y'][k]])

In [62]:
headers = ['longitude','latitude']
filename = dir_input + 'fault_infer.csv'
with open(filename,'w',newline='') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    for i in range(len(fault_infer['x'])):
        f_csv.writerow([fault_infer['x'][i],fault_infer['y'][i]])